# Empathic Zoom

Este script utiliza Selenium y la librería Deep Face para detectar las caras de los participantes en una reunión de Zoom web y mostrar las emociones de la reunión.

Para ello, el script abrirá una sesión de Chrome automatizada con Selenium, accederá a la reunión de Zoom utilizando la URL proporcionada, y luego utilizará Deep Face para analizar las expresiones faciales de los participantes.

*(Podemos llegar a usar OpenCV para analizar, ademas de las emociones, los gestos corporales)*

*Por ejemplo:*
- Detección de Movimiento y Postura: Utilizar algoritmos de visión por computadora para detectar el movimiento de las personas y su postura durante la reunión. Un aumento en el movimiento o cambios en la postura pueden indicar un mayor nivel de interés o participación.
- Reconocimiento de Gestos Faciales: Analizar los gestos faciales de los participantes para detectar sonrisas, fruncir el ceño, miradas de atención, entre otros.
- Participación en la Conversación: Analizar la participación verbal de las personas en la reunión, como la cantidad de tiempo que hablan, la frecuencia con la que hacen preguntas o comentarios, etc.

#### Librerías necesarias

- pip install selenium
- pip install deepface

In [ ]:
# Importamos librerias necesarias

from selenium import webdriver
from deepface import DeepFace
import time

In [ ]:
def capture_screenshot(driver, filename='screenshot.png'):
    driver.save_screenshot(filename)

def analyze_emotion(image_path='screenshot.png'):
    img = cv2.imread(image_path)
    return DeepFace.analyze(img, actions=['emotion'])

def compute_engagement(emotion_results):
    engagement_scores = {
        'happy': 1,
        'surprise': 1,
        'neutral': 0.5,
        'sad': -1,
        'angry': -1,
        'fear': -1,
        'disgust': -1
    }
    
    total_score = 0
    for res in emotion_results:
        dominant_emotion = res['dominant_emotion']
        total_score += engagement_scores.get(dominant_emotion, 0)
    
    return total_score / len(emotion_results)

# Initialize WebDriver
driver = webdriver.Chrome(executable_path='./chromedriver')
driver.get("https://zoom.us/wc/join/<meeting-id>")
time.sleep(30)  # Adjust based on when the meeting starts

while True:
    capture_screenshot(driver)
    emotion_result = analyze_emotion()
    engagement_score = compute_engagement([emotion_result])
    print(f"General Engagement Score: {engagement_score}")
    time.sleep(60)  # Capture screenshot every minute

driver.quit()